In [68]:
import subprocess
from tempfile import NamedTemporaryFile
from Bio.Seq import Seq
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
import pandas as pd

def bash(inCommand, autoOutfile = True):
    if autoOutfile == True:
        tmp = NamedTemporaryFile()
        subprocess.call(inCommand +' > '+tmp.name, shell=True)
        f = open(tmp.name,'r')
        tmp.close()
        return(f.read())
    else:
        subprocess.call(inCommand, shell=True)  
def BLAST(seq, BLASTtype="x", flags = 'bitscore sseqid pident qstart qend sseq qseq sstart send length slen qlen'): 
    query = NamedTemporaryFile()
    SeqIO.write(SeqRecord(Seq(seq), id="temp"), query.name, "fasta")
    tmp = NamedTemporaryFile()
    bash(
        'blast'+BLASTtype+' -query ' + query.name + ' -out ' + tmp.name +
        ' -db /Users/mattmcguffie/Downloads/swissprot/swissprot' +
        #' -db /Users/mattmcguffie/database/BLAST_dbs/fluors_20-9-2' +
        ' -culling_limit 1 ' +
        ' -max_target_seqs 1000 -outfmt "6 '+flags+'"')
    with open(tmp.name, "r") as file_handle:  #opens BLAST file
        align = file_handle.readlines()
        
    tmp.close() 
    query.close() 
    align=pd.DataFrame([ele.split() for ele in align],columns=flags.split())
    
    if not align.empty:
        #align['sseqid']=align['sseqid'].str.split("|",expand=True)[1]
        align['pident']=align['pident'].astype(float)
        align['slen']=align['slen'].astype(int)
        align['length']=align['length'].astype(int)
        align['qlen']=align['qlen'].astype(int)
        align['pmatch']=align['length']/((align['qlen']/3)-1)#because nuc->prot
        align['abspmatch']=1-abs(1-align['pmatch']) #because nuc->prot
        align['fractionmatch']=align['length']/align['slen']
        align['bitscore']=align['bitscore'].astype(float)
        align['score']=align['pident']*align['length']*align['fractionmatch']
        align=align.sort_values(by=['score'],ascending=False)
    return align 

In [64]:
csga="""atgAAACTTT TAAAAGTAGC AGCAATTGCA GCAATCGTAT TCTCCGGTAG CGCTCTGGCA
GGTGTTGTTC CTCAGTACGG CGGCGGCGGT AACCACGGTG GTGGCGGTAA TAATAGCGGC
CCAAATTCTG AGCTGAACAT TTACCAGTAC GGTGGCGGTA ACTCTGCACT TGCTCTGCAA
ACTGATGCCC GTAACTCTGA CTTGACTATT ACCCAGCATG GCGGCGGTAA TGGTGCAGAT
GTTGGTCAGG GCTCAGATGA CAGCTCAATC GATCTGACCC AACGTGGCTT CGGTAACAGC
GCTACTCTTG ATCAGTGGAA CGGCAAAAAT TCTGAAATGA CGGTTAAACA GTTCGGTGGT
GGCAACGGTG CTGCAGTTGA CCAGACTGCA TCTAACTCCT CCGTCAACGT GACTCAGGTT
GGCTTTGGTA ACAACGCGAC CGCTCATCAG TACtaa""".lower().replace(" ","").replace("\n","")

In [65]:
%time
hits=BLAST(csga)
hits

CPU times: user 7 µs, sys: 1e+03 ns, total: 8 µs
Wall time: 24.1 µs


,bitscore,sseqid,pident,qstart,qend,sseq,qseq,sstart,send,length,slen,qlen,pmatch,abspmatch,fractionmatch,score
0,261.0,sp|P28307.3|,100.0,1,453,MKLLKVAAIAAIVFSGSALAGVVPQYGGGGNHGGGGNNSGPNSELN...,MKLLKVAAIAAIVFSGSALAGVVPQYGGGGNHGGGGNNSGPNSELN...,1,151,151,151,456,1.0,1.0,1.0,15100.0


In [66]:
mTm=list(SeqIO.parse("/Users/mattmcguffie/database/plasmids/erm_gfp.fa", "fasta"))[0]
mTm=str(mTm.seq)

In [69]:
%time
BLAST(mTm)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.96 µs


,bitscore,sseqid,pident,qstart,qend,sseq,qseq,sstart,send,length,slen,qlen,pmatch,abspmatch,fractionmatch,score
0,959.0,sp|P14752.1|,97.782,3829,5316,MNIPFVVETVLHDGLLKYKFKNSKIRSITTKPGKSKGAIFAYRSKK...,MNIPFVVETVLHDGLLKYKFKNSKIRSITTKPGKSKGAIFAYRSKS...,1,496,496,496,8961,0.166109,0.166109,1.000000,48499.872000
1,498.0,sp|P0A4D5.1|,100.000,1509,2243,MNKNIKYSQNFLTSEKVLNQIIKQLNLKETDTVYEIGTGKGHLTTK...,MNKNIKYSQNFLTSEKVLNQIIKQLNLKETDTVYEIGTGKGHLTTK...,1,245,245,245,8961,0.082050,0.082050,1.000000,24500.000000
2,449.0,sp|P42212.1|,96.639,8401,7688,MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKF...,MSKGEELFTGVVPILVELDGDVNGYKFSVSGEGEGDATYGKLTLKF...,1,238,238,238,8961,0.079705,0.079705,1.000000,23000.082000
3,418.0,sp|P02981.1|,96.804,155,805,CFLMQESHKGERRPMPLRAFNPVSSFRWARGMTIVAALMTVFFIMQ...,CRYLQ--HAGERRPMPLRAFNPVSSFRWARGMTIVAALMTVFFIMQ...,178,396,219,396,8961,0.073342,0.073342,0.553030,11724.284455
8,101.0,sp|P24716.1|,84.722,2903,3118,QELEMSRSNYSRIESGKSDPTIKTLEQIVKLTNSTLVVDLI--PNE...,QELEMSRSNYSRIESGKSDPTIKTLEQIAKLTNSTLVVDLIPNEPT...,23,92,72,92,8961,0.024113,0.024113,0.782609,4773.900522
10,71.2,sp|P03851.1|,52.381,6893,7120,MHEPPVQPDRCALSGNYRLESNPVRHDLSPLAAATGNRISRARYVG...,MHEPPVQSDRCALSGNYRLESNPERHAKAPLAAATGN-----*FRG...,1,84,84,84,8961,0.028131,0.028131,1.000000,4400.004000
11,68.6,sp|P03845.1|,76.190,6983,6858,MALRRALPGWTQDDSYRIRRSGRAERGVRAHSPAWSERPTPN,VVLLHVFPGWTQDDSYRIRRSGRTERGVRAYSPAWSELPTRN,1,42,42,42,8961,0.014066,0.014066,1.000000,3199.980000
5,306.0,sp|P10021.1|,96.450,2328,2834,SVQTGK-VSSALIMGKLGSYARQNKLATALGEMGRIEKTLFTLDYI...,SFQGAKEVPSALIMGKLGSYARQNKLATALGEMGRIEKTLFTLDYI...,820,987,169,987,8961,0.056597,0.056597,0.171226,2790.991337
4,194.0,sp|P02981.1|,100.000,7390,7686,MKSNNALIVILGTVTLDAVGIGLVMPVLPGLLRDIVHSDSIASHYG...,MKSNNALIVILGTVTLDAVGIGLVMPVLPGLLRDIVHSDSIASHYG...,1,99,99,396,8961,0.033155,0.033155,0.250000,2475.000000
6,155.0,sp|P62577.1|,100.000,6028,5810,MEKKITGYTTVDISQWHRKEHFEAFQSVAQCTYNQTVQLDITAFLK...,MEKKITGYTTVDISQWHRKEHFEAFQSVAQCTYNQTVQLDITAFLK...,1,73,73,219,8961,0.024447,0.024447,0.333333,2433.333333


In [46]:
%time
BLAST(mTm)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 6.91 µs


,bitscore,sseqid,pident,qstart,qend,sseq,qseq,sstart,send,length,slen,qlen,pmatch,abspmatch,fractionmatch,score
0,877.0,tdTomato,99.790,2570,3997,MVSKGEEVIKEFMRFKVRMEGSMNGHEFEIEGEGEGRPYEGTQTAK...,MVSKGEEVIKEFMRFKVRMEGSMNGHEFEIEGEGEGRPYEGTQTAK...,1,476,476,476,8568,0.166725,0.166725,1.000000,7919.446249
1,461.0,EGFP,100.000,5090,5803,VSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKF...,VSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKF...,2,239,238,239,8568,0.083363,0.083363,0.995816,1975.726649
3,26.2,obeCFP,27.586,6601,6768,TGNIEAKYVCTSGEIPVPWATLVSTLSYGVQCFAKYPSHIKDFFKS...,SANARGEAVCVLGALPLPRSLTRCARSFG--CGERYQLTQRR*YGY...,40,97,58,234,8568,0.020315,0.020315,0.247863,8.056580
2,25.4,HriCFP,42.105,4269,4156,MPRISDKLMKTRWRGFHS-----IPSIPPDLGGIYGIG,MLRVSSSLIITRWHFF*AKQVFLIKGIPPLLPFISSIG,1,33,38,134,8568,0.013310,0.013310,0.283582,6.039118


diamond blastx -d nr -q reads.fna -o matches.m8

In [65]:
import subprocess
from tempfile import NamedTemporaryFile
from Bio.Seq import Seq
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
import pandas as pd

def bash(inCommand, autoOutfile = True):
    if autoOutfile == True:
        tmp = NamedTemporaryFile()
        subprocess.call(inCommand +' > '+tmp.name, shell=True)
        f = open(tmp.name,'r')
        tmp.close()
        return(f.read())
    else:
        subprocess.call(inCommand, shell=True)  
def dBLAST(seq, db='nr_db', flags = 'bitscore sseqid pident qstart qend sseq sstart send length slen qlen'): 
    query = NamedTemporaryFile()
    SeqIO.write(SeqRecord(Seq(seq), id="temp"), query.name, "fasta")
    tmp = NamedTemporaryFile(suffix=".m8")
    bash(
        'diamond blastx -d /Users/mattmcguffie/database/BLAST_dbs/nr -q ' + 
        query.name + ' -o ' + tmp.name + ' --outfmt 6 '+flags)
    with open(tmp.name, "r") as file_handle:  #opens BLAST file
        align = file_handle.readlines()
        
    tmp.close() 
    query.close() 
    align=pd.DataFrame([ele.split() for ele in align],columns=flags.split())
    if not align.empty:
        #align['sseqid']=align['sseqid'].str.split("|",expand=True)[1]
        align['pident']=align['pident'].astype(float)
        align['slen']=align['slen'].astype(int)
        align['length']=align['length'].astype(int)
        align['qlen']=align['qlen'].astype(int)
        align['pmatch']=align['length']/align['qlen']
        align['fractionmatch']=align['length']/align['slen']
        align['bitscore']=align['bitscore'].astype(float)
        align['score']=align['pident']*align['length'].astype(int)*align['pmatch']*align['fractionmatch']
        align=align.sort_values(by=['score'],ascending=False)
    return align 

In [79]:
%time
seq='''AATGATTTTATTTTGACTGATAGTGACCTGTTCGTTGCAACAAATTGATGAGCAATGCTTTTTTATAATG
CCAACTTTGTACAAAAAAGTTGGCACCATGAGGAGCCGGAGTAACTCCGGGGTCCGGCTGGACGGCTACG
CTCGACTGGTGCAACAGACCATCCTGTGCCATCAGAATCCAGTGACTGGCTTGCTTCCAGCCAGCTATGA
TCAGAAAGATGCTTGGGTCCGAGATAATGTGTACAGCATCTTGGCTGTGTGGGGTTTGGGCCTGGCCTAT
CGGAAGAATGCAGACCGGGATGAGGATAAGGCAAAGGCCTATGAATTGGAGCAGAGTGTAGTGAAGCTGA
TGAGAGGACTACTGCACTGCATGATCAGACAGGTGGATAAAGTAGAATCCTTCAAATATAGTCAGAGTAC
TAAGGATAGCCTCCATGCAAAGTACAACACCAAAACCTGTGCCACTGTAGTGGGTGATGATCAATGGGGA
CACCTGCAGTTGGATGCTACCTCTGTGTACCTGCTCTTCTTAGCCCAAATGACTGCCTCAGGACTCCATA
TCATCCACAGCCTAGATGAAGTCAATTTCATACAGAACCTTGTGTTTTACATTGAAGCTGCATATAAAAC
TGCTGACTTCGGGATATGGGAACGTGGAGACAAGACCAACCAAGGGATCTCAGAGTTGAATGCCAGTTCA
GTTGGAATGGCAAAGGCAGCCCTGGAAGCATTAGATGAACTGGATCTGTTTGGTGTGAAAGGTGGGCCTC
AATCAGTTATCCATGTCCTGGCTGATGAAGTACAGCACTGCCAGTCTATCCTAAATTCACTACTGCCCCG
TGCTTCACATCAAAGAGTGATGCTAGTCTACTCTCAGTGGTTTCCTTCCCTGCCTTGCAGTAGAGGG'''.lower().replace(" ","").replace("\n","")
hits=dBLAST(csga)
hits

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 19.8 µs


,bitscore,sseqid,pident,qstart,qend,sseq,sstart,send,length,slen,qlen,pmatch,fractionmatch,score
0,61.6,sp|P28307|CSGA_ECOLI,100.0,1,255,MKLLKVAAIAAIVFSGSALAGVVPQXXXXXXXXXXXXXXXPNSELN...,1,85,85,151,456,0.186404,0.562914,891.897583


In [68]:
len(hits.loc[0]['sseq'])

184